### Урок 8. ДЗ ###

1. Обучить любую модель классификации на датасете IRIS до применения самописного PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

import matplotlib.pyplot as plt

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
X, y = load_iris(return_X_y=True)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=y)
X_train.shape, X_test.shape

((120, 4), (30, 4))

In [4]:
def e_metrics(x1, x2):
    distance = np.sum(np.square(x1 - x2))
    return np.sqrt(distance)

In [5]:
def knn(x_train, y_train, x_test, k):
    
    answers = []
    for x in x_test:
        test_distances = []
            
        for i in range(len(x_train)):
            
            # расчет расстояния от классифицируемого объекта до
            # объекта обучающей выборки
            distance = e_metrics(x, x_train[i])
            
            # Записываем в список значение расстояния и ответа на объекте обучающей выборки
            test_distances.append((distance, y_train[i]))
        
        # создаем словарь со всеми возможными классами
        classes = {class_item: 0 for class_item in set(y_train)}
        
        # Сортируем список и среди первых k элементов подсчитаем частоту появления разных классов
        for d in sorted(test_distances)[0:k]:
            
            classes[d[1]] += 1

        # Записываем в список ответов наиболее часто встречающийся класс
        answers.append(sorted(classes, key=classes.get)[-1])
        
    return answers

In [6]:
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [20]:
%%time 

k = 3

y_pred = knn(X_train, y_train, X_test, k)
print(f'Точность алгоритма при k = {k}: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 3: 0.967
CPU times: user 42.6 ms, sys: 2.43 ms, total: 45 ms
Wall time: 44.7 ms


#### Применим PCA ####

In [8]:
eig_val, eig_vecs = np.linalg.eig(X.T @ X)
display(eig_val, eig_vecs)

array([9.20830507e+03, 3.15454317e+02, 1.19780429e+01, 3.55257020e+00])

array([[ 0.75110816,  0.2841749 ,  0.50215472,  0.32081425],
       [ 0.38008617,  0.5467445 , -0.67524332, -0.31725607],
       [ 0.51300886, -0.70866455, -0.05916621, -0.48074507],
       [ 0.16790754, -0.34367081, -0.53701625,  0.75187165]])

In [9]:
vec0 = eig_vecs[:, 0]
vec1 = eig_vecs[:, 1]
vec2 = eig_vecs[:, 2]
vec3 = eig_vecs[:, 3]
vec0, vec1, vec2, vec3

(array([0.75110816, 0.38008617, 0.51300886, 0.16790754]),
 array([ 0.2841749 ,  0.5467445 , -0.70866455, -0.34367081]),
 array([ 0.50215472, -0.67524332, -0.05916621, -0.53701625]),
 array([ 0.32081425, -0.31725607, -0.48074507,  0.75187165]))

In [10]:
eig_sum = sum(eig_val)
[(i / eig_sum) * 100 for i in sorted(eig_val, reverse=True)]

[96.53029806531566,
 3.3068951313646835,
 0.12556535030291374,
 0.037241453016743115]

In [11]:
X_reduced = np.dot(X, vec0)
X_reduced = np.array([i for i in zip(X_reduced, np.dot(X, vec1))])
# X_reduced

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=y)
X_train.shape, X_test.shape

((120, 2), (30, 2))

In [21]:
%%time 

y_pred = knn(X_train, y_train, X_test, k)
print(f'Точность алгоритма при k = {k}: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 3: 0.967
CPU times: user 45.4 ms, sys: 3.03 ms, total: 48.5 ms
Wall time: 51.8 ms


#### Проверка на sklearn ####

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [15]:
X, y = load_iris(return_X_y=True)

In [16]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [17]:
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=y)
X_train.shape, X_test.shape

((120, 2), (30, 2))

In [22]:
%%time 

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)
neigh.score(X_test, y_test)

CPU times: user 8.83 ms, sys: 2.41 ms, total: 11.2 ms
Wall time: 16.8 ms


0.9666666666666667

In [ ]:
# ОТВЕТ: в данном случае, используя метод мы в два раза сократили объем обрабатываемых данных, 
# что способствовало сокращению
# времени обучения, при сохранении хорошего уровня оценки.